## **MC356 - Invest Banck (INVB3)**
Dataset de índices econômicos mundiais

Integrantes:
* Leonardo Rener de Oliveira
* César Guedes Carneiro	
* Matheus Silva de Deus	

In [2]:
import csv

In [3]:
import sqlite3
from sqlite3 import Error


def create_connection():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None;
    try:
        conn = sqlite3.connect('temp_db.db')
    except Error as e:
        print(e)
    
    return conn

In [4]:
db_connection = create_connection()

cursor = db_connection.cursor()

In [5]:
%load_ext sql

In [6]:
%sql sqlite:///temp_db.db

In [7]:
%%sql
DROP TABLE IF EXISTS COUNTRY;
DROP TABLE IF EXISTS LISTED_WORLD_INDICE;
DROP TABLE IF EXISTS INDICE_FROM_COUNTRY;
DROP TABLE IF EXISTS GDP;
DROP TABLE IF EXISTS HISTORICAL_DATA;


CREATE TABLE COUNTRY (
    Name VARCHAR(100),
    PRIMARY KEY(Name)
);

CREATE TABLE LISTED_WORLD_INDICE (
    Name VARCHAR(100),
    Currency VARCHAR(3),
    PRIMARY KEY(Name)
);

CREATE TABLE INDICE_FROM_COUNTRY (
    Country VARCHAR(100),
    Indice_name VARCHAR(100),
    FOREIGN KEY (Country) REFERENCES COUNTRY,
    FOREIGN KEY (Indice_name) REFERENCES LISTED_WORD_INDICE
);

CREATE TABLE GDP (
    Country VARCHAR(100),
    Year INT,
    GDP FLOAT,
    FOREIGN KEY (Country) REFERENCES COUNTRY,
    PRIMARY KEY(Country, Year)
);

CREATE TABLE HISTORICAL_DATA (
    Indice_name VARCHAR(100),
    Year INT,
    Month INT,
    Open FLOAT,
    Close FLOAT,
    PRIMARY KEY(Indice_name, Year, Month)
);


 * sqlite:///temp_db.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

### insert data into countries table

In [8]:
with open('../data/processed/countries/countries.csv', newline='') as countries_file:
    rows = csv.reader(countries_file)
    
    next(rows, None)
    
    cursor.executemany("INSERT OR IGNORE INTO COUNTRY VALUES (?)", rows)

    db_connection.commit()

### insert data into listed world indices table

In [11]:
with open('../data/processed/listed-world-indices/listed-world-indices.csv', newline='') as listed_world_indices_file:
    rows = csv.reader(listed_world_indices_file)
    
    next(rows, None)

    cursor.executemany("INSERT OR IGNORE INTO LISTED_WORLD_INDICE VALUES (?, ?)", rows)

    db_connection.commit()

### insert data into country-indices relation table

In [12]:
with open('../data/processed/indices-from-country/indices-from-country.csv', newline='') as indices_from_country_file:
    rows = csv.reader(indices_from_country_file)
    
    next(rows, None)

    cursor.executemany("INSERT OR IGNORE INTO INDICE_FROM_COUNTRY VALUES (?, ?)", rows)

    db_connection.commit()

### insert data into GDP

In [13]:
with open('../data/processed/GDP/GDP.csv', newline='') as GDP_file:
    rows = csv.reader(GDP_file)
    cursor.executemany("INSERT OR IGNORE INTO GDP VALUES (?, ?, ?)", rows)

    db_connection.commit()

### insert data into historical data table

In [15]:
START_AT_YEAR = 2015
STOP_AT_YEAR = 2021

indices = cursor.execute('SELECT * FROM LISTED_WORLD_INDICE').fetchall()[1:]

indices_names = [indice[0].replace(' ', '_').replace('/','-') for indice in indices]

for indice_name in indices_names:
    for year in range(START_AT_YEAR, STOP_AT_YEAR):
        try:
            with open(
                '../data/processed/indices-historical-data/'+ indice_name +'/'+ str(year) + '.csv', 
                newline=''
            ) as historical_data_file:
                rows = csv.reader(historical_data_file)

                next(rows, None)

                query_data = []

                for row in rows:
                    temp_list = [
                        indice_name.replace('_', ' ').replace('-','/'),
                        year
                    ]

                    temp_list.extend(row)

                    query_data.append(temp_list)

                cursor.executemany("INSERT OR IGNORE INTO HISTORICAL_DATA VALUES (?, ?, ?, ?, ?)", query_data)
        except:
            continue

db_connection.commit()


In [23]:
import os
os.remove("temp_db.db")
print("File Removed!")

File Removed!
